In [40]:
import requests
import re

ETFs = ["XLB", "XLC", "XLY", "XLP", "XLE", "XLF", "XLV", "XLI", "XLRE", "XLK", "XLU", "SMH", "IGV", "ROBT", "TAN", "HACK", "PBW", "IBB", "AIQ", "GCLN", "SMOG", "PSL", "PXE"]
ETFs_tickers_dict = {}
ETFs_final_dict = {}
for ticker in ETFs:
    ETFs_tickers_dict[ticker] = []


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0"
}

url = "https://www.zacks.com/funds/etf/{}/holding"

with requests.Session() as req:
    req.headers.update(headers)
    for key in ETFs_tickers_dict.keys():
        r = req.get(url.format(key))
        print(f"Extracting: {r.url}")
        goal = re.findall(r'etf\\\/(.*?)\\', r.text)
        #Removing tickers with .A or .B in their names
        for ticker in goal:
            if len(ticker.split(".")) >1:
                goal.remove(ticker)
        ETFs_tickers_dict[key]+=goal


Extracting: https://www.zacks.com/funds/etf/XLB/holding
Extracting: https://www.zacks.com/funds/etf/XLC/holding
Extracting: https://www.zacks.com/funds/etf/XLY/holding
Extracting: https://www.zacks.com/funds/etf/XLP/holding
Extracting: https://www.zacks.com/funds/etf/XLE/holding
Extracting: https://www.zacks.com/funds/etf/XLF/holding
Extracting: https://www.zacks.com/funds/etf/XLV/holding
Extracting: https://www.zacks.com/funds/etf/XLI/holding
Extracting: https://www.zacks.com/funds/etf/XLRE/holding
Extracting: https://www.zacks.com/funds/etf/XLK/holding
Extracting: https://www.zacks.com/funds/etf/XLU/holding
Extracting: https://www.zacks.com/funds/etf/SMH/holding
Extracting: https://www.zacks.com/funds/etf/IGV/holding
Extracting: https://www.zacks.com/funds/etf/ROBT/holding
Extracting: https://www.zacks.com/funds/etf/TAN/holding
Extracting: https://www.zacks.com/funds/etf/HACK/holding
Extracting: https://www.zacks.com/funds/etf/PBW/holding
Extracting: https://www.zacks.com/funds/etf/I

In [68]:
import datetime
import fetcher
import pandas as pd
from finta import TA

startDate = datetime.date(2022, 12, 1)
# endDate = datetime.date(2019, 12, 15)
endDate = datetime.date.today()
for key in ETFs:

    focus_list_vol = {}
    focus_list_ADR = {}
    focus_list_RS = {}
    focus_list_RS_min_max = {}
    focus_list_SMAs = []

    index_df = fetcher.__download_data( [key], startDate, endDate, "US")
    index_df_temp = index_df.iloc[-22:]
    for ticker in ETFs_tickers_dict[key]:
        df = fetcher.__download_data([ticker], startDate, endDate, "US")

        # Greater than SMAs
        if df.tail(1)["Close"].iloc[-1] >= TA.SMA(df, period = 50).tail(1).iloc[-1]:
            focus_list_SMAs.append(ticker)
        # Volume*Price
        df_temp = df.iloc[-50:]
        avg_dollar_vol = 0
        for index, row in df_temp.iterrows():
            avg_dollar_vol += (
                df_temp.loc[index, "Volume"] * df_temp.loc[index, "Close"]
            ) / 50

        if avg_dollar_vol >= 30000000:
            focus_list_vol[ticker] = avg_dollar_vol

        # ADR%

        df_temp = df.iloc[-20:]
        sum = 0
        for index, row in df_temp.iterrows():
            sum += (df_temp.loc[index, "High"] / df_temp.loc[index, "Low"]) / 20

        ADR = 100 * (sum - 1)
        if ADR > 2:
            focus_list_ADR[ticker] = ADR

        # Relative Strength 1M
        
        df_temp = df.iloc[-22:]

        start_RS = df_temp.iloc[0]["Close"]/index_df_temp.iloc[0]["Close"]
        end_RS = df_temp.iloc[-1]["Close"]/index_df_temp.iloc[-1]["Close"]

        RS_change_perc = (end_RS-start_RS)/start_RS *100

        focus_list_RS[ticker] = RS_change_perc

        # Relative Strength low-high ratio 1M
        df_temp = df.iloc[-22:]
        RS = df_temp["Close"]/index_df_temp["Close"]
        min = RS[0]
        max = RS[1]
        for i in range(len(RS)-1):
            for j in range(i+1, len(RS)):
                if RS[j] > RS[i]:
                    if (RS[j]-RS[i])/RS[i] * 100 > (max-min)/min *100:
                        max = RS[j]
                        min = RS[i]

        RS_change_perc = (max-min)/min * 100

        focus_list_RS_min_max[ticker] = RS_change_perc
    
        focus = set(focus_list_ADR.keys()) & set(focus_list_vol.keys()) & set(focus_list_SMAs)

        # FIltering
        ADR_dict = {}
        RS_1M = {}
        RS_min_max = {}
        for ticker in focus:
            ADR_dict[ticker] = focus_list_ADR[ticker]
            RS_1M[ticker] = focus_list_RS[ticker]
            RS_min_max[ticker] = focus_list_RS_min_max[ticker]

        # Ranking by RS_min_max
        final_focus_RS_min_max = {ticker: RS for ticker, RS in sorted(RS_min_max.items(), key=lambda item: item[1], reverse=True)}
        ETFs_final_dict[key] = {}
        for i in final_focus_RS_min_max.keys():
            ETFs_final_dict[key].update({i:[final_focus_RS_min_max[i], ADR_dict[i]]})

reading from file:  yahoo_data/US/XLB2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/LIN2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/SHW2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/FCX2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/ECL2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/APD2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/NUE2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/DOW2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/NEM2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/CTVA2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/MLM2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/VMC2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/PPG2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/DD2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/LYB2022-12-012024-05-07.csv
reading from file:  yahoo_data/US/STLD2022-12-012024-05-07.csv
reading

In [64]:
key = "SMH"
for ticker in ETFs_final_dict[key].keys():
    print(ticker, ETFs_final_dict[key][ticker])

USD [16.62563585541478, 6.858733399308381]
TER [15.63662677206622, 3.2741516334363396]
MPWR [14.404960835330133, 3.8224122076980116]
NXPI [14.139968870033512, 2.9633067520430822]
OLED [12.647599836942353, 2.725266788296943]
ON [12.43066553350173, 3.6002167300225763]
MCHP [12.31842042156976, 2.7974725130661904]
TXN [12.039827999371953, 2.1943366683545795]
QCOM [11.59313182418945, 2.481668775075563]
SWKS [10.205254464552374, 2.6848857083072497]
STM [10.072986823540164, 2.0898906585686827]
NVDA [8.643691800471256, 4.7692818165896655]
ADI [8.409451858836054, 2.4499509463652425]
QRVO [8.150590788332845, 2.8567491769821984]
KLAC [6.592580792072768, 3.0359959959642913]
INTC [6.15278562597106, 2.4329110453727987]
MRVL [4.090705315044425, 3.936378068305646]
SNPS [3.8376794590453582, 2.5713436096062914]
AVGO [3.7472212530564613, 3.4268405780746525]
TSM [3.6661885498899887, 3.186301817809678]
CDNS [3.1219777569741254, 2.5687547155759605]
MU [3.0216458896450926, 3.824708259095133]
LRCX [2.52906120

In [49]:
ETFs_final_dict

{'XLB': {},
 'XLC': {},
 'XLY': {},
 'XLP': {},
 'XLE': {},
 'XLF': {},
 'XLV': {},
 'XLI': {},
 'XLRE': {},
 'XLK': {},
 'XLU': {},
 'SMH': {},
 'IGV': {},
 'ROBT': {},
 'TAN': {},
 'HACK': {},
 'PBW': {},
 'IBB': {},
 'AIQ': {},
 'GCLN': {},
 'SMOG': {},
 'PSL': {},
 'PXE': {}}

In [48]:
ETFs_final_dict[key].keys()

dict_keys([])